## 載入函式庫

In [1]:
import tensorflow as tf
from keras_segmentation.models.unet import resnet50_unet
import keras
import numpy as np
import os
import cv2
import time
import matplotlib.pyplot as plt

Using TensorFlow backend.


## 解決 TensorFlow 2.0 程式出現 cuDNN failed to initialize 錯誤問題

https://davistseng.blogspot.com/2019/11/tensorflow-2.html

In [2]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


## 建立腎臟區域預測模型

資料集：DATA
* batch_size = 2
* 腫瘤視為腎臟
* 重新調整case160的影像大小
* WW = 600, WL = 100
* Base Model: Resnet-50
* Segmentation Model: U-Net

### 查看模型架構

In [ ]:
model = resnet50_unet(n_classes = 2, input_height = 512, input_width = 512)
model.summary()

### 模型參數設定

In [5]:
dataset_dir = 'DATA\\WW600WL100'
input_size = 512
epochs = 100
batch_size = 2 # default value
steps_per_epoch = 512 # default value

### 訓練模型

目前用來評估模型表現的指標：
* average dice score per case
* average recall
* average precision
* global dice score

In [6]:
model_dir = os.path.join(dataset_dir, 'model')

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print('-----建立新資料夾：' + model_dir + '-----') 

st = time.time()

target_dataset = dataset_dir
print(f'########## 即將用該資料集進行訓練：{target_dataset} ##########')

model_name = 'resnet50_unet'
checkpoints_path = os.path.join(model_dir, model_name)

keras.backend.clear_session() # 銷毀當前的TF，避免舊模型/網路層混亂，並減緩內存
model = resnet50_unet(n_classes = 2, input_height = input_size, input_width = input_size)

model.train(
    train_images = os.path.join(dataset_dir, 'train', 'images'),
    train_annotations = os.path.join(dataset_dir, 'train', 'annotations_tumor as kidney'),
    epochs = epochs, # default = 5
    batch_size = batch_size, # default = 2
    steps_per_epoch = steps_per_epoch, # default = 512
    verify_dataset = False, # 當資料量很多時檢驗資料集會花很多時間
    checkpoints_path = checkpoints_path
)

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))
print()

-----建立新資料夾：DATA_0811_2\WW600WL100\model_resnet50_unet_bs4-----
########## 即將用該資料集進行訓練：DATA_0811_2\WW600WL100 ##########
Epoch 1/100
512/512 [==============================] - 269s 525ms/step - loss: 0.0898 - accuracy: 0.9730
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.0
Epoch 2/100
512/512 [==============================] - 241s 471ms/step - loss: 0.0099 - accuracy: 0.9969
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.1
Epoch 3/100
512/512 [==============================] - 242s 472ms/step - loss: 0.0068 - accuracy: 0.9979
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.2
Epoch 4/100
512/512 [==============================] - 241s 472ms/step - loss: 0.0044 - accuracy: 0.9986
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.3
Epoch 5/100
512/512 [==============================] - 241s 471ms/step - loss: 0.0032 - accuracy: 0.9989
saved  DATA_0811_2\WW600WL100\model_re

512/512 [==============================] - 243s 474ms/step - loss: 6.3413e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.86
Epoch 88/100
512/512 [==============================] - 242s 472ms/step - loss: 6.1149e-04 - accuracy: 0.9998
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.87
Epoch 89/100
512/512 [==============================] - 242s 472ms/step - loss: 6.0068e-04 - accuracy: 0.9998
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.88
Epoch 90/100
512/512 [==============================] - 242s 472ms/step - loss: 6.5202e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.89
Epoch 91/100
512/512 [==============================] - 242s 472ms/step - loss: 6.5960e-04 - accuracy: 0.9997
saved  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.90
Epoch 92/100
512/512 [==============================] - 242s 47

## 對測試集進行預測，並生成評估指標

In [7]:
from evaluate import *

st = time.time()

result = evaluate_model(
    image_dir = os.path.join(dataset_dir, 'test', 'images'), 
    label_dir = os.path.join(dataset_dir, 'test', 'annotations_tumor as kidney'), 
    checkpoints_path = 'DATA\\WW600WL100\\model\\resnet50_unet', # 用套件建立出來的模型儲存路徑
    calculate_predicting_indicators = True, # 是否計算並回傳評估指標
    output_predicted_result = False,  # 是否輸出模型預測結果
    segment_out_predicted_region_from_original_images = False, # 是否根據模型預測結果從原始影像中切割出預測區域 
    roi_description = 'tumor as kidney',
    preds = None)

print('測試集預測結果：')
print(f'average Dice score per case of kidney: {result[0]: .4f}')
print(f'average recall of kidney: {result[1]: .4f}')
print(f'average precision of kidney: {result[2]: .4f}')
print(f'global dice score of kidney: {result[3]: .4f}')
print('')
print('混淆矩陣：')
print(f'True Positive: {result[-4]}')
print(f'False Positive: {result[-3]}')
print(f'False Negative: {result[-2]}')
print(f'True Negative: {result[-1]}')

ed = time.time()
spend_time = ed - st
print('花費時間(秒)：' + str(spend_time))

----------生成模型預測結果----------
loaded weights  DATA_0811_2\WW600WL100\model_resnet50_unet_bs4\resnet50_unet_padding0.99


100%|████████████████████████████████████████████████████████████████████████████| 12656/12656 [14:48<00:00, 14.25it/s]


----------開始計算各項預測指標----------
目前進度：第500張照片
目前進度：第1000張照片
目前進度：第1500張照片
目前進度：第2000張照片
目前進度：第2500張照片
目前進度：第3000張照片
目前進度：第3500張照片
目前進度：第4000張照片
目前進度：第4500張照片
目前進度：第5000張照片
目前進度：第5500張照片
目前進度：第6000張照片
目前進度：第6500張照片
目前進度：第7000張照片
目前進度：第7500張照片
目前進度：第8000張照片
目前進度：第8500張照片
目前進度：第9000張照片
目前進度：第9500張照片
目前進度：第10000張照片
目前進度：第10500張照片
目前進度：第11000張照片
目前進度：第11500張照片
目前進度：第12000張照片
目前進度：第12500張照片
total case number: 12656
訓練集預測結果：
average Dice score per case of kidney:  0.9650
average recall of kidney:  0.9587
average precision of kidney:  0.9726
global dice score of kidney:  0.9605
花費時間(秒)：1585.6040244102478



In [ ]:
f = open("DATA\\patient indices of testing set - KiTS.txt", "r")
test_patient_idx = f.read().splitlines()
f.close()

dice_score_list = result[5]
print('測試集各病患的 Dice score:')
for idx, i in enumerate(dice_score_list):
    print(f'case{test_patient_idx[idx]}: {i: .4f}')